<a href="https://colab.research.google.com/github/ATalaveraDev/social-network-extraction-info/blob/main/ERS_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies

In [1]:
pip install tweepy

OAuth authentication

In [4]:
import tweepy

auth = tweepy.OAuthHandler('GJdVKUjoEl31WLHlgIRYxVmYr', 'HVaOvQWoHIkiHZh2yPNb0YMxwR6waHagtoR4hcKURo6nhvGxwL')
auth.set_access_token('1254452545583529991-g0sI2XhrtijaQ5zA3NNPRwCfuSKrCr', '4GLEprsRqsKlHPelKC4YirECbHTQYsJJGYiy0JN2OuRiH')

api = tweepy.API(auth)

[Status(_api=<tweepy.api.API object at 0x7f1f58379950>, _json={'created_at': 'Wed Feb 24 20:18:21 +0000 2021', 'id': 1364671024357326855, 'id_str': '1364671024357326855', 'text': 'Hello Soil', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1254452545583529991, 'id_str': '1254452545583529991', 'name': 'Ricardo "El Cactus"', 'screen_name': 'Ricardo70134788', 'location': '', 'description': 'Sería un cactus independiente si no viviera en una casa, por eso necesito que mi dueño sepa cuándo me tiene que regar.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Sun Apr 26 16:49:38 +000

Tweets Selection

In [9]:
def tweetsSelector(criteria):
  lat = 37.0902
  lon = 95.7129
  public_tweets = api.search(q=criteria, geolocation='37.0902,95.7129,km', count=5, lang='en')
  print(public_tweets)

tweetsSelector('biden')

[Status(_api=<tweepy.api.API object at 0x7f1f58379950>, _json={'created_at': 'Wed Feb 24 20:35:25 +0000 2021', 'id': 1364675320314400769, 'id_str': '1364675320314400769', 'text': 'RT @kylegriffin1: NBC News: The Biden administration is in the process of vetting candidates to nominate to serve on the USPS Board of Gove…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'kylegriffin1', 'name': 'Kyle Griffin', 'id': 32871086, 'id_str': '32871086', 'indices': [3, 16]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1285200781637894144, 'id_str': '1285200781637894144', 'name': 'Dr. G.', 'screen_name': 'DemNevada', 'location': 'Nevada', 'des

Sentiment Processor

In [ ]:
def sentimentProcessor:
  

sentimentProcessor()

Map